In [2]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt  # This line was missing

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

Introduction

When studying oligiopolies, one kind of monopolistic competition is called Cournot competition. In the Cournot model firms compete in quantities of output. 

In our assignment we want to analyze what happens, when two firms have an output competition. 
For simplicity, we assume constant MC for both firms. 

The price is determined by the output produced by the two firms together, via the inverse demand function. They choose their output quantity simultaneously. 

Furthermore we are assuming a linear demand curve. 


Missing text

Model description

Missing

Defining the model in python

In [12]:
#Define variables in sympy

p = sm.symbols("p") #Market price
q1 = sm.symbols("q1") #Output firm 1
q2 = sm.symbols("q2") #Output firm 2
c = sm.symbols("c") #Marginal profit
pi1 = sm.symbols("pi1") #Profit for firm 1
pi2 = sm.symbols("pi2") #Profit for firm 2
m = sm.symbols("m") #Parameter for the slope of the price function
b = sm.symbols("b") # Intercept of the price function
C1 = sm.symbols("C1") #Costs for firm 1
C2 = sm.symbols("C2") #Costs for firm 2
P = sm.symbols("P")


#Define cost functions
cost1 = sm.Eq(C1, c*q1) #Costs for firm 1
cost2 = sm.Eq(C2, c*q2) #Costs for firm 2

#Define price function
price_function = sm.Eq(P,m*(q1+q2) + b)

#Define profit for firm 1
profit1 =  m*q1**2 + m*q1*q2 + b*q1 - c*q1

#Define profit for firm 2
profit2 =  m*q2**2 + m*q1*q2 + b*q2 - c*q2


print(profit1)
print(profit2)

b*q1 - c*q1 + m*q1**2 + m*q1*q2
b*q2 - c*q2 + m*q1*q2 + m*q2**2


In Cournot firms seeks to maximize profit. Therefore, we find the best reponse function for the two firms. 

In [48]:
#First order derivative of profit for firm 1 wrt. q1 
objective1 = sm.diff(profit1, q1)

#First order derivative of profit for firm 2 wrt. q2
objective2 = sm.diff(profit2, q2)


display(objective1)
display(objective2)

#To find the best response functions, we isolate q1* and q2*
BR1 = sm.solve(objective1, q1)[0] 
BR2 = sm.solve(objective2, q2)[0]

display(BR1)
display(BR2)


b - c + 2*m*q1 + m*q2

b - c + m*q1 + 2*m*q2

(-b + c - m*q2)/(2*m)

(-b + c - m*q1)/(2*m)

A Nash Equilibrium in Cournot is where, both firms use their best reponse function. Graphically, this is where the two best response functions intersect. This is shon below. 

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Parameters
m_val, b_val, c_val = 1, 100, 10  # Example values for m, b, c

# Define the profit functions
def profit1(q1, q2):
    return -(m_val * q1**2 + m_val * q1 * q2 + b_val * q1 - c_val * q1)

def profit2(q2, q1):
    return -(m_val * q2**2 + m_val * q1 * q2 + b_val * q2 - c_val * q2)

# Define the best response functions using numerical optimization
def best_response1(q2):
    result = minimize(lambda q1: profit1(q1, q2), x0=0.1, bounds=[(0, None)])
    return result.x[0]

def best_response2(q1):
    result = minimize(lambda q2: profit2(q2, q1), x0=0.1, bounds=[(0, None)])
    return result.x[0]

# Define the function to find the equilibrium by minimizing the squared differences
def find_equilibrium(x):
    q1, q2 = x
    br1 = best_response1(q2)
    br2 = best_response2(q1)
    return (q1 - br1)**2 + (q2 - br2)**2

# Initial guess
initial_guess = [1, 1]

# Optimization to find the equilibrium
result = minimize(find_equilibrium, initial_guess, bounds=[(0, None), (0, None)])

# Output the equilibrium quantities
q1_equilibrium, q2_equilibrium = result.x
print("Equilibrium quantities:")
print("q1* =", q1_equilibrium)
print("q2* =", q2_equilibrium)

The Cournot equilibium must therefore be: ([c-b/]/m, [c-b]/m)

Equilibrium quantities:
q1* = 1.0
q2* = 1.0


/var/folders/12/264rh1fs0qdgby4_yjl2rst40000gn/T/ipykernel_74038/3719402380.py:9: RuntimeWarning: overflow encountered in square
  return -(m_val * q1**2 + m_val * q1 * q2 + b_val * q1 - c_val * q1)
/var/folders/12/264rh1fs0qdgby4_yjl2rst40000gn/T/ipykernel_74038/3719402380.py:12: RuntimeWarning: overflow encountered in square
  return -(m_val * q2**2 + m_val * q1 * q2 + b_val * q2 - c_val * q2)
